In [ ]:
import pandas as pd
from pandas import datetime
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt 
import statsmodels.api as sm
from scipy import stats
import pandas as pd
import matplotlib.pyplot as plt

import statsmodels.api as sm


In [ ]:
from statsmodels.graphics.api import qqplot

In [ ]:
def parser(x):
    return datetime.strptime(x,'%Y-%m-%d')
df = pd.read_csv('carga_gw_2020.csv', sep =';', header = None )

In [ ]:
df.head()

In [ ]:
def to_numeric(x): 
    
    x = x.replace(',','.')
    sep = x.split('.')
    
    if len(sep) == 2:
        x = sep[0]+'.'+sep[1]
        
        
    elif len(sep) ==3 :
        x = sep[0]+sep[1]+'.'+sep[2]
    
    else:
        x = 0
    
    x_num = pd.to_numeric(x)

    return  x_num

In [ ]:
def preprocess_df(df):
        
    # Determina novos nome de colunas
    df.columns = ["Date", "Carga"]
    
    df.Date = pd.to_datetime(df.Date, dayfirst= True)
    df.dtypes
    
    # Transforma colunas em numericos 
    df['Carga'] = pd.to_numeric(df['Carga'].apply(to_numeric))
    
    
    
    # Associa coluna Date ao index
    df.index = pd.to_datetime(df.Date, format="%m-%d-%Y")
    # dropa coluna Date extra 
    df.drop("Date", axis=1, inplace=True)
    
    return df

In [ ]:
def fill_na_with_mean(df):
    
    num_cols = len(list(df.columns.values))
    
    for col in range(num_cols):
        
        df.iloc[:,col] = df.iloc[:,col].fillna(df.iloc[:,col].mean())
        
    return df

In [ ]:
df = preprocess_df(df)
df = fill_na_with_mean(df)
    
df.head()

In [ ]:
df.info()

In [ ]:
df.Carga[1]

In [ ]:
df_diff = df.diff(periods=1)
# integrated of order 1, denoted by d (for diff), one of the parameters of ARIMA model

In [ ]:
df_diff.head()

In [ ]:
df_diff = df_diff[1:]

##Estacionário
significa que a média, variância e covariância são constantes para qualquer período de tempo.

Se os dados forem estacionários, então o gráfico de autocorrelacao decai rapidamente. A seguir um gráfico não estacionário.

In [ ]:
from statsmodels.graphics.tsaplots import plot_acf
plot_acf(df)

In [ ]:
plot_acf(df_diff)

In [ ]:
df_diff.plot()

In [ ]:
X = df.values
X.size

In [ ]:
train = X[0:(730)]
test = X[(730):]
predictions = []

In [ ]:
from statsmodels.tsa.ar_model import AR
from sklearn.metrics import mean_squared_error
model_ar = AR(train)
model_ar_fit = model_ar.fit()

In [ ]:
predictions = model_ar_fit.predict(start = 730, end = 932)

In [ ]:
test.size

In [ ]:
plt.plot(test)
plt.plot(predictions, color = 'red')

In [ ]:
from statsmodels.tsa.arima_model import ARIMA

In [ ]:
#p,q,d p = periods from autoregressive model
#d - integrated order, difference 
#q - periods in moving average model
model_arima = ARIMA(train,order = 7,1,1))
model_arima_fit = model_arima.fit()

In [ ]:
 predictions = model_arima_fit.forecast(steps = 182)[0]
predictions 

In [ ]:
plt.plot(test)
plt.plot(predictions, color ='red')